##  1 去掉取值变化小的特征Removing features with low variance

## 2 单变量特征选择Univariate feature selection

### 2.1 Pearson相关系数 Pearson Correlation

In [3]:
import numpy as np
from scipy.stats import pearsonr

np.random.seed(0)
size = 300
x = np.random.normal(0, 1, size)
print "Lower noise", pearsonr(x, x + np.random.normal(0, 1, size))
print "Higher noise", pearsonr(x, x + np.random.normal(0, 10, size))

Lower noise (0.7182483686213841, 7.32401731299835e-49)
Higher noise (0.057964292079338155, 0.3170099388532475)


In [5]:
x = np.random.uniform(-1, 1, 100000)
print pearsonr(x, x**2)[0]

-0.0023080470761233087


### 2.2 互信息和最大信息系数 Mutual information and maximal information coefficient (MIC)

![Image of Yaktocat](http://dataunion.org/wp-content/uploads/2015/04/4dbuAXe.png)

In [8]:
from minepy import MINE

m = MINE()
x = np.random.uniform(-1, 1, 10000)
m.compute_score(x, x**2)
print m.mic()

1.0


### 2.3 距离相关系数 (Distance correlation)

### 2.4 基于学习模型的特征排序 (Model based ranking)

In [11]:
from sklearn.cross_validation import cross_val_score, ShuffleSplit
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor

#Load boston housing dataset as an example
boston = load_boston()
X = boston["data"]
Y = boston["target"]
names = boston["feature_names"]

rf = RandomForestRegressor(n_estimators=20, max_depth=4)
scores = []
for i in range(X.shape[1]):
     score = cross_val_score(rf, X[:, i:i+1], Y, scoring="r2",
                              cv=ShuffleSplit(len(X), 3, .3))
     scores.append((round(np.mean(score), 3), names[i]))
print sorted(scores, reverse=True)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[(0.643, 'LSTAT'), (0.568, 'RM'), (0.516, 'NOX'), (0.298, 'PTRATIO'), (0.284, 'INDUS'), (0.263, 'TAX'), (0.23, 'CRIM'), (0.21, 'ZN'), (0.162, 'RAD'), (0.101, 'AGE'), (0.093, 'B'), (0.025, 'CHAS'), (-0.044, 'DIS')]


## 3 线性模型和正则化

In [26]:
from sklearn.linear_model import LinearRegression
import numpy as np

np.random.seed(0)
size = 5000

#A dataset with 3 features
X = np.random.normal(0, 1, (size, 3))
#Y = X0 + 2*X1 + noise
Y = X[:,0] + 2*X[:,1] + np.random.normal(0, 2, size)
lr = LinearRegression()
lr.fit(X, Y)

#A helper method for pretty-printing linear models
def pretty_print_linear(coefs, names = None, sort = False):
    if names == None:
        names = ["X%s" % x for x in range(len(coefs))]
    lst = zip(coefs, names)
    if sort:
        lst = sorted(lst,  key = lambda x:-np.abs(x[0]))
    return " + ".join("%s * %s" % (round(coef, 3), name)
                                   for coef, name in lst)

print "Linear model:", pretty_print_linear(lr.coef_)

 Linear model: 0.984 * X0 + 1.995 * X1 + -0.041 * X2


In [13]:
from sklearn.linear_model import LinearRegression

size = 100
np.random.seed(seed=5)

X_seed = np.random.normal(0, 1, size)
X1 = X_seed + np.random.normal(0, .1, size)
X2 = X_seed + np.random.normal(0, .1, size)
X3 = X_seed + np.random.normal(0, .1, size)

Y = X1 + X2 + X3 + np.random.normal(0,1, size)
X = np.array([X1, X2, X3]).T

lr = LinearRegression()
lr.fit(X,Y)
print "Linear model:", pretty_print_linear(lr.coef_)

Linear model: -1.291 * X0 + 1.591 * X1 + 2.747 * X2


### 3.1 正则化模型

### 3.2 L1正则化/Lasso

In [27]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_boston

boston = load_boston()
scaler = StandardScaler()
X = scaler.fit_transform(boston["data"])
Y = boston["target"]
names = boston["feature_names"]

lasso = Lasso(alpha=.3)
lasso.fit(X, Y)

print "Lasso model: ", pretty_print_linear(lasso.coef_, list(names), sort = True)

Lasso model:  -3.707 * LSTAT + 2.992 * RM + -1.757 * PTRATIO + -1.081 * DIS + -0.7 * NOX + 0.631 * B + 0.54 * CHAS + -0.236 * CRIM + 0.081 * ZN + -0.0 * INDUS + -0.0 * AGE + 0.0 * RAD + -0.0 * TAX


### 3.3 L2正则化/Ridge regression

L2正则化将系数向量的L2范数添加到了损失函数中。由于L2惩罚项中系数是二次方的，这使得L2和L1有着诸多差异，最明显的一点就是，L2正则化会让系数的取值变得平均。对于关联特征，这意味着他们能够获得更相近的对应系数。还是以Y=X1+X2为例，假设X1和X2具有很强的关联，如果用L1正则化，不论学到的模型是Y=X1+X2还是Y=2X1，惩罚都是一样的，都是2alpha。但是对于L2来说，第一个模型的惩罚项是2alpha，但第二个模型的是4*alpha。可以看出，系数之和为常数时，各系数相等时惩罚是最小的，所以才有了L2会让各个系数趋于相同的特点。

可以看出，L2正则化对于特征选择来说一种稳定的模型，不像L1正则化那样，系数会因为细微的数据变化而波动。所以L2正则化和L1正则化提供的价值是不同的，L2正则化对于特征理解来说更加有用：表示能力强的特征对应的系数是非零。

回过头来看看3个互相关联的特征的例子，分别以10个不同的种子随机初始化运行10次，来观察L1和L2正则化的稳定性。

In [32]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
size = 100

#We run the method 10 times with different random seeds
for i in range(10):
    print "Random seed %s" % i
    np.random.seed(seed=i)
    X_seed = np.random.normal(0, 1, size)
    X1 = X_seed + np.random.normal(0, .1, size)
    X2 = X_seed + np.random.normal(0, .1, size)
    X3 = X_seed + np.random.normal(0, .1, size)
    Y = X1 + 2*X2 + X3 + np.random.normal(0, 1, size)
    X = np.array([X1, X2, X3]).T


    lr = LinearRegression()
    lr.fit(X,Y)
    print "Linear model:", pretty_print_linear(lr.coef_)

    ridge = Ridge(alpha=10)
    ridge.fit(X,Y)
    print "Ridge model:", pretty_print_linear(ridge.coef_)
    print

Random seed 0
Linear model: 0.728 * X0 + 3.309 * X1 + -0.082 * X2
Ridge model: 1.234 * X0 + 1.435 * X1 + 1.176 * X2

Random seed 1
Linear model: 1.152 * X0 + 3.366 * X1 + -0.599 * X2
Ridge model: 1.289 * X0 + 1.441 * X1 + 1.037 * X2

Random seed 2
Linear model: 0.697 * X0 + 1.322 * X1 + 2.086 * X2
Ridge model: 1.268 * X0 + 1.319 * X1 + 1.38 * X2

Random seed 3
Linear model: 0.287 * X0 + 2.254 * X1 + 1.491 * X2
Ridge model: 1.217 * X0 + 1.402 * X1 + 1.315 * X2

Random seed 4
Linear model: 0.187 * X0 + 1.772 * X1 + 2.189 * X2
Ridge model: 1.247 * X0 + 1.367 * X1 + 1.386 * X2

Random seed 5
Linear model: -1.291 * X0 + 2.591 * X1 + 2.747 * X2
Ridge model: 1.054 * X0 + 1.379 * X1 + 1.433 * X2

Random seed 6
Linear model: 1.199 * X0 + 0.969 * X1 + 1.915 * X2
Ridge model: 1.311 * X0 + 1.268 * X1 + 1.377 * X2

Random seed 7
Linear model: 1.474 * X0 + 2.762 * X1 + -0.151 * X2
Ridge model: 1.312 * X0 + 1.407 * X1 + 1.195 * X2

Random seed 8
Linear model: 0.084 * X0 + 2.88 * X1 + 1.107 * X2
Ridge

## 4 随机森林

### 4.1 平均不纯度减少 mean decrease impurity

In [1]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
import numpy as np

#Load boston housing dataset as an example
boston = load_boston()
X = boston["data"]
Y = boston["target"]
names = boston["feature_names"]
rf = RandomForestRegressor()
rf.fit(X, Y)
print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), 
             reverse=True)

Features sorted by their score:
[(0.426, 'LSTAT'), (0.3826, 'RM'), (0.0604, 'DIS'), (0.033, 'CRIM'), (0.0306, 'NOX'), (0.019, 'PTRATIO'), (0.0144, 'B'), (0.012, 'TAX'), (0.0107, 'AGE'), (0.0074, 'INDUS'), (0.0027, 'RAD'), (0.0007, 'ZN'), (0.0004, 'CHAS')]


In [8]:
size = 10000
np.random.seed(seed=18)
X_seed = np.random.normal(0, 1, size)
X0 = X_seed + np.random.normal(0, .1, size)
X1 = X_seed + np.random.normal(0, .1, size)
X2 = X_seed + np.random.normal(0, .1, size)
X = np.array([X0, X1, X2]).T
Y = X0 + X1 + X2

rf = RandomForestRegressor(n_estimators=20, max_features=2)
rf.fit(X, Y);
print "Scores for X0, X1, X2:", map(lambda x:round (x,3),
                                    rf.feature_importances_)

Scores for X0, X1, X2: [0.28, 0.491, 0.229]


### 4.2 平均精确率减少 Mean decrease accuracy

In [9]:
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import r2_score
from collections import defaultdict

X = boston["data"]
Y = boston["target"]

rf = RandomForestRegressor()
scores = defaultdict(list)

#crossvalidate the scores on a number of different random splits of the data
for train_idx, test_idx in ShuffleSplit(len(X), 100, .3):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = Y[train_idx], Y[test_idx]
    r = rf.fit(X_train, Y_train)
    acc = r2_score(Y_test, rf.predict(X_test))
    for i in range(X.shape[1]):
        X_t = X_test.copy()
        np.random.shuffle(X_t[:, i])
        shuff_acc = r2_score(Y_test, rf.predict(X_t))
        scores[names[i]].append((acc-shuff_acc)/acc)
print "Features sorted by their score:"
print sorted([(round(np.mean(score), 4), feat) for
              feat, score in scores.items()], reverse=True)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Features sorted by their score:
[(0.8005, 'LSTAT'), (0.5655, 'RM'), (0.0828, 'DIS'), (0.0455, 'CRIM'), (0.04, 'NOX'), (0.0216, 'PTRATIO'), (0.0156, 'TAX'), (0.0107, 'AGE'), (0.0051, 'B'), (0.0049, 'INDUS'), (0.0022, 'RAD'), (0.0003, 'CHAS'), (0.0, 'ZN')]


## 5 两种顶层特征选择算法

### 5.1 稳定性选择 Stability selection

In [10]:
from sklearn.linear_model import RandomizedLasso
from sklearn.datasets import load_boston
boston = load_boston()

#using the Boston housing data. 
#Data gets scaled automatically by sklearn's implementation
X = boston["data"]
Y = boston["target"]
names = boston["feature_names"]

rlasso = RandomizedLasso(alpha=0.025)
rlasso.fit(X, Y)

print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 4), rlasso.scores_), 
                 names), reverse=True)

/usr/local/lib/python2.7/dist-packages/sklearn/utils/__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


Features sorted by their score:
[(1.0, 'RM'), (1.0, 'PTRATIO'), (1.0, 'LSTAT'), (0.65, 'B'), (0.55, 'CHAS'), (0.375, 'TAX'), (0.355, 'CRIM'), (0.215, 'NOX'), (0.19, 'DIS'), (0.14, 'INDUS'), (0.035, 'ZN'), (0.015, 'RAD'), (0.005, 'AGE')]


### 5.2 递归特征消除 Recursive feature elimination (RFE)

In [11]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

boston = load_boston()
X = boston["data"]
Y = boston["target"]
names = boston["feature_names"]

#use linear regression as the model
lr = LinearRegression()
#rank all features, i.e continue the elimination until the last one
rfe = RFE(lr, n_features_to_select=1)
rfe.fit(X,Y)

print "Features sorted by their rank:"
print sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), names))

Features sorted by their rank:
[(1.0, 'NOX'), (2.0, 'RM'), (3.0, 'CHAS'), (4.0, 'PTRATIO'), (5.0, 'DIS'), (6.0, 'LSTAT'), (7.0, 'RAD'), (8.0, 'CRIM'), (9.0, 'INDUS'), (10.0, 'ZN'), (11.0, 'TAX'), (12.0, 'B'), (13.0, 'AGE')]


## 6 一个完整的例子

![Image of Yaktocat](http://dataunion.org/wp-content/uploads/2015/04/ERGNG8c.png)

In [13]:
from sklearn.datasets import load_boston
from sklearn.linear_model import (LinearRegression, Ridge, 
                                  Lasso, RandomizedLasso)
from sklearn.feature_selection import RFE, f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from minepy import MINE

np.random.seed(0)

size = 750
X = np.random.uniform(0, 1, (size, 14))

#"Friedamn #1” regression problem
Y = (10 * np.sin(np.pi*X[:,0]*X[:,1]) + 20*(X[:,2] - .5)**2 +
     10*X[:,3] + 5*X[:,4] + np.random.normal(0,1))
#Add 3 additional correlated variables (correlated with X1-X3)
X[:,10:] = X[:,:4] + np.random.normal(0, .025, (size,4))

names = ["x%s" % i for i in range(1,15)]

ranks = {}

def rank_to_dict(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x, 2), ranks)
    return dict(zip(names, ranks ))

lr = LinearRegression(normalize=True)
lr.fit(X, Y)
ranks["Linear reg"] = rank_to_dict(np.abs(lr.coef_), names)

ridge = Ridge(alpha=7)
ridge.fit(X, Y)
ranks["Ridge"] = rank_to_dict(np.abs(ridge.coef_), names)


lasso = Lasso(alpha=.05)
lasso.fit(X, Y)
ranks["Lasso"] = rank_to_dict(np.abs(lasso.coef_), names)


rlasso = RandomizedLasso(alpha=0.04)
rlasso.fit(X, Y)
ranks["Stability"] = rank_to_dict(np.abs(rlasso.scores_), names)

#stop the search when 5 features are left (they will get equal scores)
rfe = RFE(lr, n_features_to_select=5)
rfe.fit(X,Y)
ranks["RFE"] = rank_to_dict(map(float, rfe.ranking_), names, order=-1)

rf = RandomForestRegressor()
rf.fit(X,Y)
ranks["RF"] = rank_to_dict(rf.feature_importances_, names)


f, pval  = f_regression(X, Y, center=True)
ranks["Corr."] = rank_to_dict(f, names)

mine = MINE()
mic_scores = []
for i in range(X.shape[1]):
    mine.compute_score(X[:,i], Y)
    m = mine.mic()
    mic_scores.append(m)

ranks["MIC"] = rank_to_dict(mic_scores, names)


r = {}
for name in names:
    r[name] = round(np.mean([ranks[method][name] 
                             for method in ranks.keys()]), 2)

methods = sorted(ranks.keys())
ranks["Mean"] = r
methods.append("Mean")

print "\t%s" % "\t".join(methods)
for name in names:
    print "%s\t%s" % (name, "\t".join(map(str, 
                         [ranks[method][name] for method in methods])))

	Corr.	Lasso	Linear reg	MIC	RF	RFE	Ridge	Stability	Mean
x1	0.3	0.79	1.0	0.39	0.55	1.0	0.77	0.77	0.7
x2	0.44	0.83	0.56	0.61	0.67	1.0	0.75	0.72	0.7
x3	0.0	0.0	0.5	0.34	0.13	1.0	0.05	0.0	0.25
x4	1.0	1.0	0.57	1.0	0.56	1.0	1.0	1.0	0.89
x5	0.1	0.51	0.27	0.2	0.29	0.78	0.88	0.55	0.45
x6	0.0	0.0	0.02	0.0	0.01	0.44	0.05	0.0	0.07
x7	0.01	0.0	0.0	0.07	0.02	0.0	0.01	0.0	0.01
x8	0.02	0.0	0.03	0.05	0.01	0.56	0.09	0.0	0.1
x9	0.01	0.0	0.0	0.09	0.01	0.11	0.0	0.0	0.03
x10	0.0	0.0	0.01	0.04	0.0	0.33	0.01	0.0	0.05
x11	0.29	0.0	0.6	0.43	0.39	1.0	0.59	0.37	0.46
x12	0.44	0.0	0.14	0.71	0.35	0.67	0.68	0.46	0.43
x13	0.0	0.0	0.48	0.23	0.07	0.89	0.02	0.0	0.21
x14	0.99	0.16	0.0	1.0	1.0	0.22	0.95	0.62	0.62


## 总结

## tip